In [1]:
import pandas as pd
import os
import re

In [2]:
facility="SCP"
dir_path = r'/home/trishalaswain/SCP_oct_xlsx/'.format(facility)
res = []

for path in os.listdir(dir_path):
    if path.startswith('.'):
        continue
    if path.endswith('EANDM.xlsx') or path.endswith('CPT.xlsx') or path.endswith('CM.xlsx') or path.endswith('HCPCS.xlsx'):
        continue
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(r'/home/trishalaswain/SCP_oct_xlsx/'.format(facility)+path)

In [3]:
df=[]
for r in res:
    df.append(pd.read_excel(r))

In [4]:
# cleaning of suggestion and final codes

def clean(x):
    x=str(x)
    new_code=''
    if re.match('.*9921[1-5].*',x) and re.match('.*9920[1-5].*',x):
            codes=x.split(',')
            for code in codes:
                if re.match('.*9921[1-5].*', code):
                    new_code=code.strip()   
    
    if re.match('.*992[0-9]{2}.*',x) and re.match('.*994[0-9]{2}.*',x):
            codes=x.split(',')
            for code in codes:
                if re.match('.*992[0-9]{2}.*', code):
                    new_code=code.strip()   
               
    if new_code!='':
        codes=re.findall('\d{5}',new_code)
    else:
        codes=re.findall('\d{5}',x)
    codes.sort()
    
    if codes!=[]:
        return codes[-1]
    else:
        return '[]'
    
for df_new in df:
    df_new['new_sugg'] = df_new.EANDM_Suggested.apply(clean)
    df_new['new_final'] = df_new.EANDM_Final.apply(clean)

In [5]:
# df_new.EANDM_Final

In [6]:
# category 1 - we did not suggest any code

def get_category_1(x):
    sugg = x.new_sugg
    final = str(x.new_final)
    
    # a
    if sugg == "{}" and re.match('.*(992[018]{1}5).*',final):
        return "category_1_a"

    # b
    elif sugg == "{}" and re.match('.*(9929[12]).*',final):
        return "category_1_b"

    # c
    elif sugg == "{}" and re.match('.*(9944[1-3]).*',final):
        return "category_1_c"

    # d
    elif sugg == "{}" and re.match('.*(993[8-9][1-7]).*',final):
        return "category_1_d"

    # e
    elif sugg == "{}" and re.match('\d.*',final):
        return "category_1_e"
    
    return "null"

for df_new in df:
    df_new['category1'] = df_new.apply(get_category_1, axis=1)

In [7]:
# for df_new in df:
#     print(df_new.category1.value_counts())
#     print(df_new.facilityName.unique())

In [8]:
# category 2 a - missed the place of care

def get_category_2a(x):
    sugg = x.new_sugg
    final = str(x.new_final)
    
    # i
    if re.match('.*9928[1-5].*',sugg) and re.match('.*9929[12].*',final):
        return "category_2a_i"

    # ii
    elif re.match('.*9929[12].*',sugg) and re.match('.*9928[1-5].*',final):
        return "category_2a_ii"

    # iii
    elif re.match('.*9944[1-3].*',sugg) and re.match('.*992[01][2-5].*',final):
        return "category_2a_iii"

    # iv
    elif re.match('.*992[01][2-5].*',sugg) and re.match('.*9944[1-3].*',final):
        return "category_2a_iv"

    # v
    elif re.match('.*992[01][2-5].*',sugg) and re.match('.*993[89][1-7].*',final):
        return "category_2a_v"

    # vi
    elif re.match('.*993[89][1-7].*',sugg) and re.match('.*992[01][2-5].*',final):
        return "category_2a_vi"

    return "null"

for df_new in df:
    df_new['category2a'] = df_new.apply(get_category_2a, axis=1)
    

In [9]:
# for d in df:
#     print(d.category2a.value_counts())
#     print(df_new.facilityName.unique())

In [10]:
# category 2 b - missed the place of care

def get_category_2b(x):
    sugg = x.new_sugg
    final = str(x.new_final)
    
    # i
    if re.match('.*9920[1-5].*',sugg) and re.match('.*9921[1-5].*',final):
        return "category_2b_i"
    # ii
    elif re.match('.*9921[1-5].*',sugg) and re.match('.*9920[1-5].*',final):
        return "category_2b_ii"

    # iii
    elif re.match('.*9938[1-7].*',sugg) and re.match('.*9939[1-7].*',final):
        return "category_2b_iii"

    # iv
    elif re.match('.*9939[1-7].*',sugg) and re.match('.*9938[1-7].*',final):
        return "category_2b_iv"

    return "null"


for df_new in df:
    df_new['category2b'] = df_new.apply(get_category_2b, axis=1)

In [11]:
# for df_new in df:
#     print(df_new.category2b.value_counts())
#     print(df_new.facilityName.unique())

In [12]:
# category 2 c - missed the place of care

def get_category_2c(x):
    sugg = x.new_sugg
    final = str(x.new_final)

    # i
  
    if re.match('.*9928[1-4].*',sugg) and re.match('.*99285.*',final):
        return "category_2c_i"

    # ii
    elif re.match('.*99285.*',sugg) and re.match('.*9928[1-4].*',final):
        return "category_2c_ii"

    # iii
    elif re.match('.*992[01][1-4].*',sugg) and re.match('.*992[01]5.*',final):
        return "category_2c_iii"

    # iv
    elif re.match('.*992[01]5.*',sugg) and re.match('.*992[01][1-4].*',final):
        return "category_2c_iv"
    
    # v
    elif (sugg=='99291' and final=='99292') or (sugg=='99292' and final=='99291'): 
        return "category_2c_v"

    return "null"

for df_new in df:
    df_new['category2c'] = df_new.apply(get_category_2c, axis=1)

In [13]:
# print(re.match('.*992[01][1-4].*','99214, 99215') and re.match('.*992[01]5.*','99215'))

In [14]:
# for df_new in df:
#     print(df_new.category2c.value_counts())
#     print(df_new.facilityName.unique())

In [15]:

def get_match(sug, fin):

    if sug==fin:
        return True
    return False


def get_not_accepted(x):
    sugg = x.new_sugg
    final = str(x.new_final)

    # a they didn't accept any code
    if re.match('\d.*',sugg) and final=='[]':
        return "category_3a"

    # b the exact match code
    elif get_match(sugg, final):
        return "category_3b"

    return "other"

for df_new in df:
    # print(df_new.facilityName.unique())
    df_new['category3'] = df_new.apply(get_not_accepted, axis=1)

In [ ]:
def get_final_category(x):
    cat1=x.category1
    cat2a=x.category2a
    cat2b=x.category2b
    cat2c=x.category2c
    cat3=x.category3
    
    if cat1!="null":
        return cat1
        
    if cat2a!="null":
        return cat2a
    
    if cat2b!="null":
        return cat2b
    
    if cat2c!="null":
        return cat2c
    
    if cat3!="null":
        return cat3
        
    return "null"


for df_new in df:
    df_new['final_category'] = df_new.apply(get_final_category, axis=1)

In [ ]:
# for df_new in df:
#     print(df_new.category3.value_counts())
#     print(df_new.facilityName.unique())

In [ ]:
# where category 1,2a,2b,2c and category 3 conflict

# for df_new in df:
#     for d1, d2a, d2b, d2c, d3, dAcc, fn in zip(df_new.category1,df_new.category2a,df_new.category2b,df_new.category2c,df_new.category3,df_new.accountNumber,df_new.facilityName):
#         if (d1!='null' or d2a!='null' or d2b!='null' or d2c!='null') and d3!='other':
#             print(dAcc,fn)

In [ ]:
# category 1: When we did not suggest any code:
# 	a) How many level 5 codes
# 	b) how many critical care codes: 91, 92
# 	c) How many telephone codes: 441 - 443
# 	d) preventive 381-387 & 91-97
# 	e) How many other codes

# Category 2: We suggested wrong code:
# 	a) how many times we missed the place of care
# 		i) We suggested 81-85, they added 91, 92
# 		ii) We suggested 91, 92, they added 81-85
# 		iii) We suggested 441 - 443, they added 212-215
# 		iv) We suggested 212-215, they added 441, 443
# 		v) we suggested 212-215 and they added 381-387 & 391-397
# 		vi) we suggested 381-387 & 391-397 and they added 212-215
# 	b) How many visit types we mismatched
# 		i) We suggested NP 201 - 205, they added EP 212 - 215
# 		ii) We suggested EP 212 - 215, they added NP 201 - 205
# 		iii) we suggested NP 381-387 and they added 391-397
# 		iv) we suggested NP 391-397 and they added 381-387
# 	c) How many level 5 mismatch in same place of care
# 		i) We suggested 81 - 84, they added 85
# 		ii) We suggested 85, they added 81 - 84
# 		iii) We suggested 212-214 or 201-204, they added 205 or 215
# 		iv) We suggested 205 or 215, they added 212-214 or 201-204
# 		v) miss-match with 91 and 92

# Category 3: final result
# 	a) they didn't accept any code
# 	b) they accepted our code

In [ ]:
# code range of facilities

def get_code_range(x):
    # sugg = x.new_sugg
    final = str(x.new_final)
                
    if final=='nan' or final=='[]':
        return 'other'

    code=int(final)
    
    if code>=99201 and code<=99205:
        return '99202-99205'
    
    if code>=99211 and code<=99215:
        return '99212-99215'
    
    if code>=99281 and code<=99285:
        return '99281-99285'
    
    if code>=99291 and code<=99292:
        return '99291-99292'
    
    if code>=99221 and code<=99223:
        return '99221-99223'
    
    if code>=99231 and code<=99233:
        return '99231-99233'
    
    if code>=99234 and code<=99236:
        return '99234-99236'
    
    if code>=99238 and code<=99239:
        return '99238-99239'
    
    if code>=99381 and code<=99387:
        return '99381-99387'
    
    if code>=99391 and code<=99397:
        return '99224-99226'
    
    if code>=99251 and code<=99255:
        return '99251-99255'
    
    if code>=99218 and code<=99220:
        return '99218-99220'
    
    if code>=99224 and code<=99226:
        return '99224-99226'
    
    if code==99217:
        return '99217'
    
    return 'other'

for df_new in df:
    df_new['code_range']=df_new.apply(get_code_range, axis=1)

In [ ]:
# df_new

In [ ]:
# for df_new in df:
#     display(df_new.code_range)

In [ ]:
# find code ranges per facility

code_ranges=[]

for df_new in df:
    # df_code.append(df_new.groupby(df_new.code_range))
    code_ranges.append(df_new.code_range.unique())

In [ ]:
# code_ranges

In [ ]:
# df_new

In [ ]:
for code_ran in df_new.code_range.unique(): 
    temp = df_new[df_new.code_range==code_ran]
    to_take = 20
    if to_take>temp.shape[0]:
        to_take = temp.shape[0]
    
    display(temp.sample(to_take).accountNumber)

615    85186079
419    85109670
590    85197674
102    84754220
82     84614155
256    84928168
252    84910984
50     84772368
346    85168735
57     84772350
451    85079871
677    85295753
394    84979200
33     84754213
64     84772370
637    85261746
485    85030295
290    84928172
155    84842832
518    85079877
Name: accountNumber, dtype: int64

195    84868096
219    84874628
275    84910998
542    84669003
315    84687159
498    85046132
389    84964324
540    84687144
472    84996752
274    84910996
670    85279236
536    84687147
539    85094911
164    84825509
482    84996754
37     84717732
81     84772366
336    84717729
Name: accountNumber, dtype: int64

In [ ]:
def check_row(x):
    if x.accountNumber==7763806:
        print(x)

for df_new in df:
    df_new.apply(check_row, axis=1)

In [ ]:
df_new=pd.concat(df)
df_new.loc[:, ~df_new.columns.isin(['new_sugg', 'new_final'])].to_csv('{}_SEPT_COMBINED_new.csv'.format(facility), index=False)

In [ ]:
# for df_new,r in zip(df, res):
#     fname=r.rsplit('/',1)[1].rsplit('.xlsx')[0]+'.csv'
#     # df_new.style.set_properties(**{'text-align': 'left'})
#     df_new.to_csv('{}'.format(fname), index=False)

In [ ]:
# df_new.facilityName.unique()

In [ ]:
print(added_count)

NameError: name 'added_count' is not defined